In [1]:
import cobra
import json

In [5]:
def get_enzyme_constraint_model(json_model_file):
    """using enzyme concentration constraint
    json model to create a COBRApy model.

    Arguments
    ----------
    * json_model_file: json Model file.

    :return: Construct an enzyme-constrained model.
    """

    dictionary_model = json_load(json_model_file)
    model = cobra.io.json.load_json_model(json_model_file)
    prot_pool = cobra.Metabolite('prot_pool')
    prot_pool.compartment = 'c'
    coefficients = dict()
    for rxn in model.reactions:
        for eachr in dictionary_model['reactions']:
            if rxn.id == eachr['id']:
                if eachr['kcat_MW']:
                    rxn.add_metabolites({prot_pool:-1/float(eachr['kcat_MW'])})
                    # coefficients[rxn.forward_variable] = 1 / float(eachr['kcat_MW'])
                break

    lowerbound = dictionary_model['enzyme_constraint']['lowerbound']
    upperbound = dictionary_model['enzyme_constraint']['upperbound']
    
    R_pool = cobra.Reaction('enzyme_pool')
    R_pool.add_metabolites({prot_pool:1})
    R_pool.lower_bound = lowerbound
    R_pool.upper_bound = upperbound
    model.add_reactions([R_pool])
    #print(upperbound)
    # constraint = model.problem.Constraint(0, lb=lowerbound, ub=upperbound)
    # model.add_cons_vars(constraint)
    # model.solver.update()
    # constraint.set_linear_coefficients(coefficients=coefficients)
    return model
def get_enzyme_constraint_model_original(json_model_file):
    """using enzyme concentration constraint
    json model to create a COBRApy model.

    Arguments
    ----------
    * json_model_file: json Model file.

    :return: Construct an enzyme-constrained model.
    """

    dictionary_model = json_load(json_model_file)
    model = cobra.io.json.load_json_model(json_model_file)
    # prot_pool = cobra.Metabolite('prot_pool')
    # prot_pool.compartment = 'c'
    coefficients = dict()
    for rxn in model.reactions:
        for eachr in dictionary_model['reactions']:
            if rxn.id == eachr['id']:
                if eachr['kcat_MW']:
                    # rxn.add_metabolites({prot_pool:-1/float(eachr['kcat_MW'])})
                    coefficients[rxn.forward_variable] = 1 / float(eachr['kcat_MW'])
                break

    lowerbound = dictionary_model['enzyme_constraint']['lowerbound']
    upperbound = dictionary_model['enzyme_constraint']['upperbound']
    
    # R_pool = cobra.Reaction('enzyme_pool')
    # R_pool.add_metabolites({prot_pool:1})
    # R_pool.lower_bound = lowerbound
    # R_pool.upper_bound = upperbound
    # model.add_reactions([R_pool])
    #print(upperbound)
    constraint = model.problem.Constraint(0, lb=lowerbound, ub=upperbound)
    model.add_cons_vars(constraint)
    model.solver.update()
    constraint.set_linear_coefficients(coefficients=coefficients)
    return model

def json_load(path):
    """Loads the given JSON file and returns it as dictionary.

    Arguments
    ----------
    * path: The path of the JSON file
    """
    with open(path) as f:
        dictionary = json.load(f)
    return dictionary

In [7]:
fn = '../../models/c_glutamicum/smoment_iCW773.xml'
model = cobra.io.read_sbml_model(fn)

Set parameter Username
Academic license - for non-commercial use only - expires 2024-02-26


In [8]:
for r in model.boundary:
    if r.id[-7:] == 'reverse':
        # print(r, r.upper_bound)
        r_forward = model.reactions.get_by_id(r.id[:-8])
        if r.upper_bound != 0:
            r_forward.lower_bound = -r.upper_bound
            # print(r_forward,r_forward.name, r_forward.bounds)
        r.remove_from_model()

In [9]:
cobra.io.write_sbml_model(model, fn)

In [10]:
model.metabolites.r5p_e.reactions

frozenset()